<a href="https://colab.research.google.com/github/merucode/RL/blob/35-Study-Doc-Custom_ENV_grid_world/02_Reinforce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone -b 35-Study-Doc-Custom_ENV_grid_world https://github.com/merucode/RL.git
!cd RL && mv * ../

Cloning into 'RL'...
remote: Enumerating objects: 192, done.
remote: Counting objects: 100% (112/112), done.
remote: Compressing objects: 100% (111/111), done.
remote: Total 192 (delta 43), reused 0 (delta 0), pack-reused 80
Receiving objects: 100% (192/192), 11.42 MiB | 5.12 MiB/s, done.
Resolving deltas: 100% (70/70), done.


In [3]:
!pip install -e gym-examples

Obtaining file:///content/gym-examples
  Preparing metadata (setup.py) ... done
  Attempting uninstall: gym-examples
    Found existing installation: gym-examples 0.0.1
    Uninstalling gym-examples-0.0.1:
      Successfully uninstalled gym-examples-0.0.1
  Running setup.py develop for gym-examples


### NOTE: After inistall gym-examples, Restart runtime

In [1]:
import time
import matplotlib.pyplot as plt
import gym
import numpy as np

In [2]:
env = gym.make("gym_examples:gym_examples/GridWorld-v0")

SyntaxError: ignored

In [8]:
# single env
print(f'observation space : {env.observation_space}')
print(f'action space      : {env.action_space}')
print(f'action shape      : {env.action_space.n}')

observation space : Dict('agent': Box(0, 4, (2,), int64), 'target': Box(0, 4, (2,), int64))
action space      : Discrete(4)
action shape      : 4


In [32]:
obs, _ = env.reset()
print(obs)

obs = np.hstack((obs['agent'], obs['target']))
print(obs)

{'agent': array([4, 2]), 'target': array([1, 2])}
[4 2 1 2]


In [11]:
action = env.action_space.sample()
env.step(0)

({'agent': array([3, 2]), 'target': array([3, 0])},
 0,
 False,
 False,
 {'distance': 2.0})

# Reinforce model

In [14]:
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
import numpy as np

#Device
device = device = 'cuda' if torch.cuda.is_available() else 'cpu'

#Hyperparameters
learning_rate = 0.0002
gamma         = 0.98

class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.data = []

        self.fc1 = nn.Linear(4, 128)
        self.fc2 = nn.Linear(128, 4)
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x), dim=0)
        return x

    def put_data(self, item):
        self.data.append(item)

    def train_net(self):
        R = 0
        self.optimizer.zero_grad()
        for r, prob in self.data[::-1]:
            R = r + gamma * R
            loss = -torch.log(prob) * R
            loss.backward()
        self.optimizer.step()
        self.data = []

def main():
    #env = gym.make('CartPole-v1')
    env = gym.make("gym_examples:gym_examples/GridWorld-v0")
    pi = Policy()
    score = 0.0
    print_interval = 100

    pi.to(device)

    for n_epi in range(6000):
        #s, _ = env.reset()
        obs, _ = env.reset()
        s = np.hstack((obs['agent'], obs['target']))
        done = False

        while not done:
            prob = pi(torch.from_numpy(s).float().to(device))
            m = Categorical(prob)
            a = m.sample()
            obs_next, r, done, truncated, info = env.step(a.item())
            s_prime =  np.hstack((obs_next['agent'], obs_next['target']))
            pi.put_data((r,prob[a]))
            s = s_prime
            score += r

        pi.train_net()

        if n_epi%print_interval==0 and n_epi!=0:
            print("# of episode :{}, avg score : {}".format(n_epi, score/print_interval))
            score = 0.0
    env.close()

    return pi

In [15]:
if __name__ == '__main__':
    pi = main()

# of episode :100, avg score : 0.384899999999985
# of episode :200, avg score : 0.6324000000000227
# of episode :300, avg score : 0.6109000000000189
# of episode :400, avg score : 0.6935000000000039
# of episode :500, avg score : 0.7153000000000084
# of episode :600, avg score : 0.7833999999999941
# of episode :700, avg score : 0.7645999999999935
# of episode :800, avg score : 0.7544999999999926
# of episode :900, avg score : 0.8318999999999848
# of episode :1000, avg score : 0.8369999999999933
# of episode :1100, avg score : 0.8461999999999843
# of episode :1200, avg score : 0.8586999999999931
# of episode :1300, avg score : 0.8765999999999944
# of episode :1400, avg score : 0.859499999999988
# of episode :1500, avg score : 0.8850999999999984
# of episode :1600, avg score : 0.8813999999999939
# of episode :1700, avg score : 0.8800999999999881
# of episode :1800, avg score : 0.8989999999999896
# of episode :1900, avg score : 0.9005999999999914
# of episode :2000, avg score : 0.87619999

# Render

In [11]:
!apt-get install -y xvfb
!pip install pyvirtualdisplay

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libfontenc1 libxfont2 libxkbfile1 x11-xkb-utils xfonts-base xfonts-encodings
  xfonts-utils xserver-common
The following NEW packages will be installed:
  libfontenc1 libxfont2 libxkbfile1 x11-xkb-utils xfonts-base xfonts-encodings
  xfonts-utils xserver-common xvfb
0 upgraded, 9 newly installed, 0 to remove and 16 not upgraded.
Need to get 7,812 kB of archives.
After this operation, 11.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfontenc1 amd64 1:1.1.4-1build3 [14.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxfont2 amd64 1:2.0.5-1build1 [94.5 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxkbfile1 amd64 1:1.1.0-1build3 [71.8 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/main amd64 x11-xkb-utils amd64 7.7+5build4 [172 kB]
Get:5 http://archiv

In [12]:
from pyvirtualdisplay import Display
Display(visible=False, size=(1400, 900)).start()

In [20]:
from gym.wrappers import RecordVideo
from IPython.display import HTML
from base64 import b64encode

def test_env(env_name, policy): #, obs_rms
    env = gym.make(env_name)
    env = RecordVideo(env, 'videos', episode_trigger=lambda e: True)
    pi = policy
    pi.to(device)

    for episode in range(10):
        obs, _ = env.reset()
        s = np.hstack((obs['agent'], obs['target']))
        score = 0
        done = False

        while not done:
            prob = pi(torch.from_numpy(s).float().to(device))
            m = Categorical(prob)
            a = m.sample()
            obs_next, r, done, truncated, info = env.step(a.item())
            s_prime =  np.hstack((obs_next['agent'], obs_next['target']))
            score += r
        print(f'episode {episode} score : {score}')
    del env


def display_video(episode=0):
    video_file = open(f'/content/videos/rl-video-episode-{episode}.mp4', "r+b").read()
    video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
    return HTML(f"")

In [21]:
test_env("gym_examples:gym_examples/GridWorld-v0", pi)

episode 0 score : 0.92
episode 1 score : 0.96
episode 2 score : 0.98
episode 3 score : 0.97
episode 4 score : 0.89
episode 5 score : 0.2699999999999996
episode 6 score : 0.21999999999999953
episode 7 score : 0.99
episode 8 score : 0.6099999999999999
episode 9 score : 0.98
